**BUILDING A MACHINE LEARNING PIPELINE ON STARTUPS ACQUISITION**

**STEPS INVOLVED**


*   Importing Dataset
*   Data Cleaning
*   Data Labelling
*   Data Encoding
*   Exploratory Data Analysis
*   Feature Engineering
*   Model Building
*   Pipeline
*   Model Deployment




**IMPORTING NECESSARY LIBRARIES**

In [1]:
import numpy as np
import pandas as pd

#scikit-learn class that helps to take care of missing values
from sklearn.impute import SimpleImputer

# to handle outliers
from scipy.stats import zscore

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
import warnings
warnings.filterwarnings('ignore')


# for standardizing the features
from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer



**IMPORTING DATASET**

In [2]:
company = pd.read_csv('companies.csv')
company.head()

,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,...,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,...,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.606209,-122.332071,15.5
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,...,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.021122,-118.396467,NaN
2,c:100,2,Company,100,NaN,There,there,/company/there,games_video,acquired,...,2003-02-01,2011-09-23,4.0,12.0,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,37.562992,-122.325525,NaN
3,c:10000,3,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN,NaN,NaN
4,c:10001,4,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN,NaN,NaN


In [3]:
company.describe()

,Unnamed: 0.1,entity_id,parent_id,logo_width,logo_height,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,lat,lng,ROI
count,196553.000000,196553.000000,0.0,110110.000000,110110.000000,2591.000000,2591.000000,31707.000000,2.787400e+04,91699.000000,129667.000000,83852.000000,83852.000000,726.000000
mean,98276.000000,153006.227333,NaN,459.132685,222.728917,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,37.564512,-52.123066,45.745037
std,56740.108067,90209.250941,NaN,594.982577,333.090722,12.173510,11.436955,1.201666,6.775937e+07,0.540099,9.100309,15.477102,70.049067,572.035638
min,0.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,2.910000e+02,1.000000,1.000000,-50.942326,-159.497746,0.011111
25%,49138.000000,59850.000000,NaN,192.000000,70.000000,1.000000,1.000000,1.000000,5.000000e+05,1.000000,1.000000,34.052234,-111.940005,2.648879
50%,98276.000000,174539.000000,NaN,267.000000,105.000000,1.000000,1.000000,1.000000,2.564500e+06,1.000000,1.000000,39.768403,-77.036871,6.500497
75%,147414.000000,232655.000000,NaN,484.000000,232.000000,1.000000,1.000000,2.000000,1.100000e+07,1.000000,3.000000,45.421530,-0.127758,13.549900
max,196552.000000,286215.000000,NaN,18200.000000,9600.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,77.553604,176.212549,13333.333333


**DATA CLEANING**


1.   Deleting irrelevant and redundant information

  *   Delete 'region','city' as they provide too much of granularity.
  *   Delete 'id', 'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 'created_by', 'created_at', 'updated_at' as they are redundant.
  *   Delete 'domain', 'homepage_url', 'twitter_username', 'logo_url', 'logo_width', 'logo_height', 'short_description', 'description', 'overview','tag_list', 'name', 'normalized_name', 'permalink', 'invested_companies' as they are irrelevant features.
    *   Delete duplicate values if found any.

  *   Delete those which has more than 98% of null values.



2.   Delete instances with missing values for 'status', 'country_code', 'category_code' and 'founded_at'.
(Since these are the type of data where adding value via imputation will create wrong pattern only)
3. Delete outliers for 'funding_total_usd' and 'funding_rounds'.

*   Find the IQR
*   Find the Upper and Lower limit

*   Find outliers and drop it
*   Delete contradictory (mutually opposed or inconsistent data).








In [4]:
# checking for the null values
company.isnull().sum().sort_values(ascending=False)

parent_id              196553
ROI                    195827
first_investment_at    193970
last_investment_at     193970
investment_rounds      193962
invested_companies     193962
closed_at              193933
short_description      189422
funding_total_usd      168679
last_funding_at        165046
first_funding_at       165046
funding_rounds         164846
state_code             145650
twitter_username       115962
tag_list               115101
lat                    112701
lng                    112701
city                   112663
country_code           108563
founded_at             105326
first_milestone_at     104854
milestones             104854
last_milestone_at      104854
description            104505
logo_height             86443
logo_width              86443
logo_url                86443
category_code           73367
homepage_url            70008
domain                  70008
overview                69582
relationships           66886
created_by              41020
normalized

In [5]:
# deleting 'region', 'city', 'state_code', as they provide too much of granularity
company.drop(columns=['state_code','city','region'], inplace=True)

# deleting 'id',  'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 'created_by','created_at', 'updated_at' as they are redundant.
company.drop(columns=['id',  'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 'created_by','created_at', 'updated_at'],inplace=True)

# Delete 'domain', 'homepage_url', 'twitter_username', 'logo_url', 'logo_width', 'logo_height', 'short_description', 'description', 'overview','tag_list', 'name', 'normalized_name', 'permalink', 'invested_companies' as they are irrelevant features.
company.drop(columns=['domain', 'homepage_url', 'twitter_username', 'logo_url', 'logo_width', 'logo_height', 'short_description', 'description', 'overview','tag_list', 'name', 'normalized_name', 'permalink', 'invested_companies'], inplace=True)

In [6]:
# check if there are any duplicates
print(company.duplicated().any())

# delete duplicate values if any

print(company.drop_duplicates(inplace=True))

print(company.duplicated().sum())

print(company.isnull().sum().sort_values(ascending=False))

True
None
0
ROI                    108738
first_investment_at    107217
last_investment_at     107217
investment_rounds      107213
closed_at              106845
funding_total_usd       81602
first_funding_at        77992
last_funding_at         77992
funding_rounds          77793
first_milestone_at      53353
last_milestone_at       53353
milestones              53353
relationships           34403
lat                     28363
lng                     28363
founded_at              26913
country_code            24870
category_code           12230
status                      0
dtype: int64


In [7]:
#calculate the percentage of null values for each columns
print('percantage of null values in each column\n')
print((company.isnull().mean()*100).sort_values(ascending=False))

percantage of null values in each column

ROI                    99.336768
first_investment_at    97.947270
last_investment_at     97.947270
investment_rounds      97.943616
closed_at              97.607433
funding_total_usd      74.546883
first_funding_at       71.248995
last_funding_at        71.248995
funding_rounds         71.067200
first_milestone_at     48.740225
last_milestone_at      48.740225
milestones             48.740225
relationships          31.428598
lat                    25.910802
lng                    25.910802
founded_at             24.586165
country_code           22.719798
category_code          11.172623
status                  0.000000
dtype: float64


In [8]:
# dropping the columns that has more than 98% null values
company.drop('ROI', axis=1, inplace=True)

# Since we can see only nan values so let's check how much of ros has nan values.
print(company.isnull().sum().sort_values(ascending=False))

first_investment_at    107217
last_investment_at     107217
investment_rounds      107213
closed_at              106845
funding_total_usd       81602
last_funding_at         77992
first_funding_at        77992
funding_rounds          77793
last_milestone_at       53353
milestones              53353
first_milestone_at      53353
relationships           34403
lat                     28363
lng                     28363
founded_at              26913
country_code            24870
category_code           12230
status                      0
dtype: int64


In [9]:
# Define the columns to check for missing values
columns_to_check = ['status', 'country_code', 'category_code', 'founded_at']
company.dropna(subset=columns_to_check, inplace=True)

# Since we can see only nan values so et's check how much of rows has nan values.
company.isnull().sum().sort_values(ascending=False)

first_investment_at    62837
last_investment_at     62837
investment_rounds      62836
closed_at              61924
funding_total_usd      43118
last_funding_at        40836
first_funding_at       40836
funding_rounds         40696
last_milestone_at      28341
milestones             28341
first_milestone_at     28341
relationships          15290
lat                     2374
lng                     2374
status                     0
country_code               0
founded_at                 0
category_code              0
dtype: int64

In [10]:
# Finding IQR
# For funding_total_usd
check_ftu = 'funding_total_usd'

#calculate Q1 and Q3
Q1 = company[check_ftu].quantile(0.25)
Q3 = company[check_ftu].quantile(0.75)
IQR_ftu = Q3 - Q1
print("Inter-quartile range for funding_total_usd : ", IQR_ftu)

# Find upper-limit and lower-limit
lower_limit_ftu = Q1 - 1.5 * IQR_ftu
upper_limit_ftu = Q3 + 1.5 * IQR_ftu

print(f"Lower Limit for {check_ftu}: {lower_limit_ftu }")
print(f"Upper Limit for {check_ftu}: {upper_limit_ftu}")


Inter-quartile range for funding_total_usd :  11488962.0
Lower Limit for funding_total_usd: -16722405.0
Upper Limit for funding_total_usd: 29233443.0


In [11]:
# For funding_total_usd
outliers_ftu = company[(company[check_ftu]< lower_limit_ftu)| (company[check_ftu]> upper_limit_ftu)]
outliers_ftu

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng
0,web,operating,2005-10-17,NaN,USA,NaN,NaN,NaN,2005-10-01,2008-05-19,3.0,39750000.0,2010-09-05,2013-09-18,5.0,17.0,47.606209,-122.332071
19,health,operating,2007-10-01,NaN,USA,NaN,NaN,NaN,2008-10-10,2013-08-13,5.0,68069200.0,NaN,NaN,NaN,14.0,37.774929,-122.419415
24,analytics,operating,2006-01-01,NaN,USA,NaN,NaN,NaN,2008-08-25,2013-03-28,3.0,33000000.0,2010-05-18,2013-03-01,3.0,19.0,37.774929,-122.419415
66,mobile,acquired,2005-01-01,NaN,USA,NaN,NaN,NaN,2005-03-01,2012-07-02,7.0,121500000.0,2012-01-19,2013-08-13,3.0,45.0,42.360082,-71.058880
80,games_video,operating,2003-04-01,NaN,USA,NaN,NaN,NaN,2006-08-28,2012-01-01,3.0,95097200.0,2009-11-12,2013-01-01,3.0,28.0,37.774929,-122.419415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196290,mobile,operating,2001-01-01,NaN,DEU,NaN,NaN,NaN,2002-02-13,2009-12-15,5.0,40673570.0,2002-02-01,2002-02-01,1.0,11.0,48.027728,11.586918
196298,hardware,operating,1995-01-01,NaN,USA,NaN,NaN,NaN,2008-08-19,2010-12-21,3.0,146000000.0,2009-05-01,2010-07-15,3.0,23.0,37.485215,-122.236355
196415,security,operating,1997-01-01,NaN,USA,2010-12-08,2010-12-08,1.0,2005-02-07,2005-02-07,1.0,108000000.0,2011-07-01,2013-02-21,3.0,17.0,39.920541,-105.086650
196496,games_video,acquired,2004-01-01,NaN,CHN,NaN,NaN,NaN,2005-01-01,2011-02-04,4.0,286500000.0,2009-05-01,2011-02-04,2.0,3.0,31.230390,121.473702


In [12]:
# Drop outliers
company.drop(outliers_ftu.index, inplace=True)

In [13]:
# For funding_rounds
check_fr = 'funding_rounds'
#calculate Q1 and Q3
Q1 = company[check_fr].quantile(0.25)
Q3 = company[check_fr].quantile(0.75)
IQR_fr = Q3 - Q1
print("Inter-quartile range for funding_rounds : ", IQR_fr)

# Find upper-limit and lower-limit
lower_limit_fr = Q1 - 1.5 * IQR_fr
upper_limit_fr = Q3 + 1.5 * IQR_fr
print(f"Lower Limit for {check_fr}: {lower_limit_fr }")
print(f"Upper Limit for {check_fr}: {upper_limit_fr}")

Inter-quartile range for funding_rounds :  1.0
Lower Limit for funding_rounds: -0.5
Upper Limit for funding_rounds: 3.5


In [14]:
# Drop outliers
company.drop(company[(company['funding_rounds']< lower_limit_fr)| (company['funding_rounds']> upper_limit_fr)].index, inplace=True)

In [15]:
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59987 entries, 5 to 196552
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_code        59987 non-null  object 
 1   status               59987 non-null  object 
 2   founded_at           59987 non-null  object 
 3   closed_at            1544 non-null   object 
 4   country_code         59987 non-null  object 
 5   first_investment_at  663 non-null    object 
 6   last_investment_at   663 non-null    object 
 7   investment_rounds    664 non-null    float64
 8   first_funding_at     19151 non-null  object 
 9   last_funding_at      19151 non-null  object 
 10  funding_rounds       19291 non-null  float64
 11  funding_total_usd    16880 non-null  float64
 12  first_milestone_at   32423 non-null  object 
 13  last_milestone_at    32423 non-null  object 
 14  milestones           32423 non-null  float64
 15  relationships        44871 non-null  flo

**DATA TRANSFORMATION**



1.   Changes in Original data


    *   Convert founded_at, closed_at, first_funded_at, last_funding_at, first_milestone_at ,
       last_milestone_at to years.
    *   Generalize the categorical data i.e. category_code, status and category_code.


2.   Create new variables


    *   Create new feature isClosed from closed_at and status.
    *   reate new feature 'active_days'






In [16]:
#Convert founded_at, closed_at, first_funded_at, last_funding_at, first_milestone_at ,last_milestone_at to years.
# founded_at
company['founded_at'] = company['founded_at'].fillna('2021')
company['founded_at'] = company['founded_at'].str[0:4].astype(int)

# closed_at
company['closed_at'] = company['closed_at'].fillna('2021')
company['closed_at'] = company['closed_at'].str[0:4].astype(int)

# first_funding_at
company['first_funding_at'] = company['first_funding_at'].fillna('2021')
company['first_funding_at'] = company['first_funding_at'].str[0:4].astype(int)

# last_funding_at
company['last_funding_at'] = company['last_funding_at'].fillna('2021')
company['last_funding_at'] = company['last_funding_at'].str[0:4].astype(int)

# first_milestone_at
company['first_milestone_at'] = company['first_milestone_at'].fillna('2021')
company['first_milestone_at'] = company['first_milestone_at'].str[0:4].astype(int)

# last_milestone_at
company['last_milestone_at'] = company['last_milestone_at'].fillna('2021')
company['last_milestone_at'] = company['last_milestone_at'].str[0:4].astype(int)

company

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng
5,advertising,operating,2007,2021,MAR,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,2.0,30.427755,-9.598107
6,cleantech,operating,2008,2021,IND,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,NaN,22.307159,73.181219
12,advertising,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,2.0,35.686975,-105.937799
13,web,acquired,2007,2021,USA,NaN,NaN,NaN,2008,2008,1.0,5000000.0,2008,2012,3.0,14.0,37.386052,-122.083851
15,games_video,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,3.0,33.078655,-116.601964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,ecommerce,operating,2007,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2013,2013,2.0,5.0,37.774929,-122.419415
196549,public_relations,operating,2007,2021,USA,NaN,NaN,NaN,2008,2008,1.0,750000.0,2013,2013,1.0,14.0,37.338208,-121.886329
196550,consulting,operating,1959,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2012,2013,3.0,44.0,38.882334,-77.171091
196551,search,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,1.0,34.052234,-118.243685


In [17]:
# Generalize the categorical data i.e. category_code, status and category_code.
# 1. category_code
print(company['category_code'].nunique())
print(company['country_code'].nunique())

# Lets Check the repetition of value in ascending order and keep the first 10 values and name
# remaining one as other.
column_name = 'category_code'
value_counts = company[column_name].value_counts()
print(value_counts)

42
165
category_code
software            10888
web                  7752
ecommerce            4831
other                4700
mobile               3700
advertising          3575
games_video          2789
consulting           2780
enterprise           2472
biotech              2006
public_relations     1561
hardware             1434
network_hosting      1254
education            1149
search                976
cleantech             798
analytics             646
social                645
finance               638
security              612
health                601
legal                 362
medical               361
semiconductor         360
travel                330
news                  282
manufacturing         276
fashion               271
hospitality           267
real_estate           220
local                 197
photo_video           196
messaging             180
music                 179
sports                165
transportation        126
nonprofit             114
design           

In [18]:
top_10_values_category = value_counts.head(10)
print("Top 10 values in category :\n",top_10_values_category)

# remaining categories
Other_values = company[~company[column_name].isin(top_10_values_category.index)][column_name]

# Lets keep the category as same of it falls under above 10 otherwise let's replace it with other.
company.loc[Other_values.index,column_name] = "Other"

Top 10 values in category :
 category_code
software       10888
web             7752
ecommerce       4831
other           4700
mobile          3700
advertising     3575
games_video     2789
consulting      2780
enterprise      2472
biotech         2006
Name: count, dtype: int64


In [19]:
# Let's check if we've more than 15 columns
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59987 entries, 5 to 196552
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_code        59987 non-null  object 
 1   status               59987 non-null  object 
 2   founded_at           59987 non-null  int32  
 3   closed_at            59987 non-null  int32  
 4   country_code         59987 non-null  object 
 5   first_investment_at  663 non-null    object 
 6   last_investment_at   663 non-null    object 
 7   investment_rounds    664 non-null    float64
 8   first_funding_at     59987 non-null  int32  
 9   last_funding_at      59987 non-null  int32  
 10  funding_rounds       19291 non-null  float64
 11  funding_total_usd    16880 non-null  float64
 12  first_milestone_at   59987 non-null  int32  
 13  last_milestone_at    59987 non-null  int32  
 14  milestones           32423 non-null  float64
 15  relationships        44871 non-null  flo

In [20]:
# Lets Check the repetition of value in ascending order and keep the first 10 values and name
# remaining one as other.
col_name = 'country_code'
value_counts_country = company[col_name].value_counts()

# top 10 values
top_10_values_country = value_counts_country.head(10)
top_10_values_country

country_code
USA    33844
GBR     4959
IND     2985
CAN     2609
DEU     1358
FRA     1226
AUS     1027
ESP      880
ISR      764
NLD      653
Name: count, dtype: int64

In [21]:
# remaining country_code
Other_values_country = company[~company[col_name].isin(top_10_values_country.index)][col_name]

# Lets keep the country as same of it falls under above 10 otherwise let's replace it with other.
company.loc[Other_values_country.index,col_name] = "Other"

In [22]:
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59987 entries, 5 to 196552
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_code        59987 non-null  object 
 1   status               59987 non-null  object 
 2   founded_at           59987 non-null  int32  
 3   closed_at            59987 non-null  int32  
 4   country_code         59987 non-null  object 
 5   first_investment_at  663 non-null    object 
 6   last_investment_at   663 non-null    object 
 7   investment_rounds    664 non-null    float64
 8   first_funding_at     59987 non-null  int32  
 9   last_funding_at      59987 non-null  int32  
 10  funding_rounds       19291 non-null  float64
 11  funding_total_usd    16880 non-null  float64
 12  first_milestone_at   59987 non-null  int32  
 13  last_milestone_at    59987 non-null  int32  
 14  milestones           32423 non-null  float64
 15  relationships        44871 non-null  flo

**CREATE NEW VARIABLES**

a. Create new feature isClosed from closed_at and status.


    *   if the value in status is 'operating' or 'ipo', Let's put 1.
    *  Where as if the value is 'acquired' or 'closed', let's put 0.

b. Create new feature 'active_days'

1.   Replacing values:


    *   if the value in status is 'operating' or 'ipo' in closed_at, Let's put 2021.
    *    Where as if the value is 'acquired' or 'closed', let's put 0.

2.   Subtract founded_date from closed_date, and calculate age in days (After calculating active days,
     check contradictory issues we didn't check it before).

3. Then, delete the closed_at column.



In [23]:
# Creating new column isClosed
company['isClosed'] = ((company['status'] != 'acquired') & (company['status'] != 'closed')).astype(int)
company.head()

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng,isClosed
5,advertising,operating,2007,2021,Other,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,2.0,30.427755,-9.598107,1
6,Other,operating,2008,2021,IND,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,NaN,22.307159,73.181219,1
12,advertising,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,2.0,35.686975,-105.937799,1
13,web,acquired,2007,2021,USA,NaN,NaN,NaN,2008,2008,1.0,5000000.0,2008,2012,3.0,14.0,37.386052,-122.083851,0
15,games_video,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,3.0,33.078655,-116.601964,1


In [24]:
# Replacing values at closed_at according to the values in status
company['closed_at'] = np.where(company['status'].str.contains('acquired|closed', case=False, na=False), 0, company['closed_at'].fillna(2021))
company['closed_at']

5         2021
6         2021
12        2021
13           0
15        2021
          ... 
196548    2021
196549    2021
196550    2021
196551    2021
196552    2021
Name: closed_at, Length: 59987, dtype: int32

In [25]:
company['founded_at']

5         2007
6         2008
12        2008
13        2007
15        2008
          ... 
196548    2007
196549    2007
196550    1959
196551    2008
196552    2007
Name: founded_at, Length: 59987, dtype: int32

In [26]:
# subtracting founded_date from closed_date ans calculating active_days
company['active_days'] = (company['closed_at'] - company['founded_at'])*365

In [27]:
# company.info()
company['active_days']

5           5110
6           4745
12          4745
13       -732555
15          4745
           ...  
196548      5110
196549      5110
196550     22630
196551      4745
196552      5110
Name: active_days, Length: 59987, dtype: int32

In [28]:
index= np.where(company['active_days']<0)
company['active_days']= company['active_days'].drop(company['active_days'].index[index])

company['active_days'] = company['active_days'].fillna(0)

In [29]:
company.head()

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng,isClosed,active_days
5,advertising,operating,2007,2021,Other,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,2.0,30.427755,-9.598107,1,5110.0
6,Other,operating,2008,2021,IND,NaN,NaN,NaN,2021,2021,NaN,NaN,2021,2021,NaN,NaN,22.307159,73.181219,1,4745.0
12,advertising,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,2.0,35.686975,-105.937799,1,4745.0
13,web,acquired,2007,0,USA,NaN,NaN,NaN,2008,2008,1.0,5000000.0,2008,2012,3.0,14.0,37.386052,-122.083851,0,0.0
15,games_video,operating,2008,2021,USA,NaN,NaN,NaN,2021,2021,NaN,NaN,2008,2008,1.0,3.0,33.078655,-116.601964,1,4745.0


In [30]:
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59987 entries, 5 to 196552
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_code        59987 non-null  object 
 1   status               59987 non-null  object 
 2   founded_at           59987 non-null  int32  
 3   closed_at            59987 non-null  int32  
 4   country_code         59987 non-null  object 
 5   first_investment_at  663 non-null    object 
 6   last_investment_at   663 non-null    object 
 7   investment_rounds    664 non-null    float64
 8   first_funding_at     59987 non-null  int32  
 9   last_funding_at      59987 non-null  int32  
 10  funding_rounds       19291 non-null  float64
 11  funding_total_usd    16880 non-null  float64
 12  first_milestone_at   59987 non-null  int32  
 13  last_milestone_at    59987 non-null  int32  
 14  milestones           32423 non-null  float64
 15  relationships        44871 non-null  flo

In [31]:
# delete closed_at column
company.drop('closed_at', inplace=True, axis=1)

#####**Let's work on target variable now.**

In [32]:
# Remove the null values with the mean value in 'Numerical Data'.
# Imputing using Mean
company['funding_rounds'].fillna(company['funding_rounds'].mean(), inplace=True)
company['funding_total_usd'].fillna(company['funding_total_usd'].mean(), inplace=True)
company['investment_rounds'].fillna(company['investment_rounds'].mean(), inplace=True)
company['milestones'].fillna(company['milestones'].mean(), inplace=True)
company['relationships'].fillna(company['relationships'].mean(), inplace=True)
company['lat'].fillna(company['lat'].mean(), inplace=True)
company['lng'].fillna(company['lng'].mean(), inplace=True)

In [33]:
company['funding_total_usd'] = pd.to_numeric(company['funding_total_usd'], errors='coerce')
company['funding_total_usd'] = company['funding_total_usd'].apply(lambda x: '{:,.0f}'.format(x))
company['funding_total_usd'] = company['funding_total_usd'].str.replace(',','').astype(float)

In [34]:
# Imputing using mode for categorical data.
company['first_investment_at'].fillna(company['first_investment_at'].mode().iloc[0], inplace=True)
company['last_investment_at'].fillna(company['last_investment_at'].mode().iloc[0], inplace=True)

In [35]:
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59987 entries, 5 to 196552
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_code        59987 non-null  object 
 1   status               59987 non-null  object 
 2   founded_at           59987 non-null  int32  
 3   country_code         59987 non-null  object 
 4   first_investment_at  59987 non-null  object 
 5   last_investment_at   59987 non-null  object 
 6   investment_rounds    59987 non-null  float64
 7   first_funding_at     59987 non-null  int32  
 8   last_funding_at      59987 non-null  int32  
 9   funding_rounds       59987 non-null  float64
 10  funding_total_usd    59987 non-null  float64
 11  first_milestone_at   59987 non-null  int32  
 12  last_milestone_at    59987 non-null  int32  
 13  milestones           59987 non-null  float64
 14  relationships        59987 non-null  float64
 15  lat                  59987 non-null  flo

In [36]:
# Saving the csv file
company.to_csv('cleaned_companies.csv', index=False)

In [37]:
company

,category_code,status,founded_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng,isClosed,active_days
5,advertising,operating,2007,Other,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2021,2021,1.356845,2.000000,30.427755,-9.598107,1,5110.0
6,Other,operating,2008,IND,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2021,2021,1.356845,3.891288,22.307159,73.181219,1,4745.0
12,advertising,operating,2008,USA,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2008,2008,1.000000,2.000000,35.686975,-105.937799,1,4745.0
13,web,acquired,2007,USA,2010-01-01,2012-01-01,4.373494,2008,2008,1.00000,5000000.0,2008,2012,3.000000,14.000000,37.386052,-122.083851,0,0.0
15,games_video,operating,2008,USA,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2008,2008,1.000000,3.000000,33.078655,-116.601964,1,4745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,ecommerce,operating,2007,USA,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2013,2013,2.000000,5.000000,37.774929,-122.419415,1,5110.0
196549,Other,operating,2007,USA,2010-01-01,2012-01-01,4.373494,2008,2008,1.00000,750000.0,2013,2013,1.000000,14.000000,37.338208,-121.886329,1,5110.0
196550,consulting,operating,1959,USA,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2012,2013,3.000000,44.000000,38.882334,-77.171091,1,22630.0
196551,Other,operating,2008,USA,2010-01-01,2012-01-01,4.373494,2021,2021,1.42538,4561781.0,2021,2021,1.356845,1.000000,34.052234,-118.243685,1,4745.0
